In [80]:
import pandas as pd
import numpy as np
import csv
import sklearn
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.metrics import r2_score, mean_absolute_percentage_error, mean_squared_error, mean_absolute_error
from scipy.stats import pearsonr
import tensorflow as tf
from tensorflow import keras

In [81]:
data=pd.read_excel("createsdataset.xlsx")
data.to_csv('createsdataset.csv',index=True)
data=data.dropna()


In [82]:
print(data.head)

<bound method NDFrame.head of      Temperature©[min]  D.O. (mg/L)[max]  pH[min]  \
0                 17.0               8.6     8.40   
1                 30.0               7.8     8.30   
2                 33.0               6.0     8.20   
3                 34.0               7.0     8.60   
4                 34.0               7.3     8.20   
..                 ...               ...      ...   
134               20.0               7.1     7.30   
135               24.0               7.4     6.90   
136               19.0               8.1     6.10   
137               21.0               7.5     7.20   
138               16.0               8.2     6.78   

     Conductivity(μSiemens/cm)[min]  BOD (mg/L)[min]  Total coliform[min]  
0                             542.0             2.70               9000.0  
1                             492.0             2.80               4600.0  
2                             750.0             2.90               5000.0  
3                            

In [85]:
train, test = train_test_split(data,test_size=0.1,random_state=100)
train_features = train.loc[:,data.columns !='D.O. (mg/L)[max]']
test_features = test.loc[:,data.columns !='D.O. (mg/L)[max]']
train_label = train['D.O. (mg/L)[max]']
test_label = test['D.O. (mg/L)[max]']

In [86]:
print(train_features.shape)
print(train_label.shape)
print(test_features.shape)
print(test_label.shape)

(111, 5)
(111,)
(13, 5)
(13,)


In [87]:
train_features = preprocessing.MinMaxScaler(feature_range=(0, 1)).fit_transform(train_features)
test_features = preprocessing.MinMaxScaler(feature_range=(0, 1)).fit_transform(test_features)

In [88]:
# ANFIS MODEL CODE
from Models import myanfis

In [10]:
import sys
sys.maxsize

9223372036854775807

In [105]:
param = myanfis.fis_parameters(
    n_input=5,                # no. of Regressors
    n_memb=10,                 # no. of fuzzy memberships
    batch_size=1,            # 16 / 32 / 64 / ...
    memb_func='gaussian',      # 'gaussian' / 'gbellmf'
    optimizer='sgd',          # sgd / adam / ...
    # mse / mae / huber_loss / mean_absolute_percentage_error / ...
    loss='mse',
    n_epochs=150               # 10 / 25 / 50 / 100 / ...
)

In [106]:
fis = myanfis.ANFIS(n_input=param.n_input,
                    n_memb=param.n_memb,
                    batch_size=param.batch_size,
                    memb_func=param.memb_func,
                    name='myanfis'
                    )

# fis = myanfis.ANFIS(n_input=6,
#                     n_memb=3,
#                     batch_size=1,
#                     memb_func='gaussian',
#                     name='myanfis'
#                     )
# compile model
fis.model.compile(optimizer=param.optimizer,
                    loss=param.loss,
                    metrics=['mse'])

In [107]:
histories=[]
history = fis.fit(train_features, train_label,
                    epochs=param.n_epochs,
                    batch_size=param.batch_size,
                    validation_data=(test_features, test_label)
                    
                    )
histories.append(history)

Epoch 1/150
111/111 [==============================] - 2s 8ms/step - loss: 7593.8633 - mse: 7593.8633 - val_loss: 728.5397 - val_mse: 728.5397
Epoch 2/150
111/111 [==============================] - 1s 6ms/step - loss: 7585.8784 - mse: 7585.8784 - val_loss: 711.5109 - val_mse: 711.5109
Epoch 3/150
111/111 [==============================] - 1s 5ms/step - loss: 7552.5581 - mse: 7552.5581 - val_loss: 665.2390 - val_mse: 665.2390
Epoch 4/150
111/111 [==============================] - 1s 6ms/step - loss: 7444.8481 - mse: 7444.8481 - val_loss: 622.9696 - val_mse: 622.9696
Epoch 5/150
111/111 [==============================] - 1s 6ms/step - loss: 7441.8687 - mse: 7441.8687 - val_loss: 610.2928 - val_mse: 610.2928
Epoch 6/150
111/111 [==============================] - 1s 6ms/step - loss: 7423.7788 - mse: 7423.7788 - val_loss: 605.8282 - val_mse: 605.8282
Epoch 7/150
111/111 [==============================] - 1s 6ms/step - loss: 7425.8530 - mse: 7425.8530 - val_loss: 592.8421 - val_mse: 592.8421